In [2]:

from langchain_community.vectorstores import FAISS
import os
from pyprojroot import here
from langchain_openai import OpenAIEmbeddings  # We'll replace this
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from pprint import pprint
load_dotenv()

True

In [3]:
os.environ['GOOGLE_API_KEY'] = os.getenv("GOOGLE_API_KEY")

# Use Google's embedding model
EMBEDDING_MODEL = "models/embedding-001"  
VECTORDB_DIR = "data/airline_policy_vectordb_faiss"
K = 2

In [4]:
embedding_function = GoogleGenerativeAIEmbeddings(model=EMBEDDING_MODEL)

In [5]:
try:
    vectordb = FAISS.load_local(
        folder_path=str(here(VECTORDB_DIR)),
        embeddings=embedding_function
    )
    print("Loaded existing FAISS index")
except:
    print("No existing FAISS index found. Please create one first.")

No existing FAISS index found. Please create one first.


In [6]:
print("Number of vectors in vectordb:", len(vectordb.index_to_docstore_id), "\n\n")

NameError: name 'vectordb' is not defined